<a href="https://colab.research.google.com/github/YujiSue/BioInfoTools/blob/master/HumanDisease2Celegans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ortholog and disease DB Trial

In [ ]:
#@title Download dataset
# Sqlite3 DB
!curl -L -o ortho.db "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/ortho.db?alt=media&token=103137df-052d-4c07-b780-9ec745b58cd6"
!curl -L -o disease.db "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/disease.db?alt=media&token=44bfb52d-72cb-46f8-97fb-31e79affae74"
# C. elegans genome
!curl -L -o nematode.bin "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/nematode.bin?alt=media&token=b5048edd-161f-47b7-91e1-3eb50be04ac3"
!curl -L -o nematode.db "https://firebasestorage.googleapis.com/v0/b/publicstorage-3ef6a.appspot.com/o/nematode.db?alt=media&token=8cea64e6-39d7-4f6d-b02e-b2cd9cd382ae"

In [2]:
#@title Load database
import sqlite3
ocon = sqlite3.connect('/content/ortho.db')
ocur = ocon.cursor()
dcon = sqlite3.connect('/content/disease.db')
dcur = dcon.cursor()
cecon = sqlite3.connect('/content/nematode.db')
cecur = cecon.cursor()

In [ ]:
#@title Supported species
from IPython.display import HTML
sql = f"select * from taxon"
ocur.execute(sql)
taxons = ocur.fetchall()
dataset = []
for taxon in taxons:
  dataset.append(f'<tr><td>{taxon[0]}</td><td>{taxon[1]}</td><td>{taxon[2]}</td></tr>')
html = f'<table><tr><th>Taxon ID</th><th>Label</th><th>Species</th></tr>{"".join(dataset)}</table>'
display(HTML(html))

Taxon ID,Label,Species
6239,Nematode,Caenorhabditis elegans
7227,Fruit fly,Drosophila melanogaster
7955,Zebrafish,Danio rerio
8355,Common clawed frog,Xenopus laevis
8364,Western clawed frog,Xenopus tropicalis
9606,Human,Homo sapiens
10090,Mouse,Mus musculus
10116,Rat,Rattus norvegicus
559292,Yeast,Saccharomyces cerevisiae


In [ ]:
#@title Gene ID conversion
#@markdown Set query for conversion to _C. elegans_ gene ID.<br>
#@markdown Query format should be accroding to the below.<br>
#@markdown - Human : Gene symbol (i.e. GAPDH) or HGNC prefixed ID (i.e. HGNC:4141)
#@markdown - Mouse : Gene symbol (i.e. Gapdh) or MGI prefixed ID (i.e. MGI:95640)
#@markdown - Rat : Gene symbol (i.e. Gapdh) or RGD prefixed ID (i.e. RGD:2661)
#@markdown - Zebrafish : Gene symbol (i.e. gapdh) or ZFIN prefixed ID (i.e. ZDB-GENE-030115-1)
#@markdown - Fruit fly : Gene symbol (i.e. Gapdh1) or Flybase prefixed ID (i.e. FBgn0001091)
query = '' #@param {type: 'string'}
sql = f"SELECT gene2,gid1,gene1 from orthology where taxon1=6239 and (gid2 like '{query}' or gene2 like '{query}%') group by gid1 UNION SELECT gene1,gid2,gene2 from orthology where taxon2=6239 and (gid1 like '{query}' or gene1 like '{query}%') group by gid2"
ocur.execute(sql)
genes = ocur.fetchall()
if len(genes) == 0:
  print("Not found.")
else:
  for gene in genes:
    print(gene[0],gene[1],gene[2])



In [ ]:
#@title Gene ID conversion (from file)
#@markdown Prepare gene symbol or prefixed ID list<br>
#@markdown Check [sample file](https://github.com/YujiSue/BioInfoTools/blob/master/genelist_sample.txt)
import pandas as pd
from IPython.display import HTML
import sqlite3
file = '' #@param {type: 'string'}
df = pd.read_csv(file, header=None)
results = []
for idx, row in df.iterrows():
  query = row[0]
  sql = f"SELECT gid2,gene2,gid1,gene1 from orthology where taxon1=6239 and (gid2 like '{query}' or gene2 like '{query}%') UNION SELECT gid1,gene1,gid2,gene2 from orthology where taxon2=6239 and (gid1 like '{query}' or gene1 like '{query}%')"
  ocur.execute(sql)
  genes = ocur.fetchall()
  if len(genes) == 0:
    results.append((query,"", "Not found.", ""))
  else:
    for gene in genes:
      results.append((gene[0], gene[1], gene[2], gene[3]))

# Output result
content = []
for res in results:
  content.append(f'<tr><td style="border:1px solid black;">{res[0]}</td><td style="border:1px solid black;">{res[1]}</td><td style="border:1px solid black;">{res[2]}</td><td style="border:1px solid black;">{res[3]}</td></tr>')
html = f'<table style="border-collapse: collapse;"><tr><th style="border:1px solid black;">Query ID</th><th style="border:1px solid black;">Query Gene</th><th style="border:1px solid black;">CE Gene ID</th><th style="border:1px solid black;">CE Gene Name</th></tr>{"".join(content)}</table>'
display(HTML(html))

In [ ]:
#@title Search c. elegans genes related to human disease
import re
import sqlite3
from IPython.display import HTML
#@markdown Set disease name or keywords<br>
#@markdown (ex. "type 1 diabetes")
query = '' #@param {type:'string'}
#
qlist = []
words = query.split(' ')
for wrd in words:
  enc = wrd.replace("'", "''")
  qlist.append(f"name like '%{enc}%'")
#
sql = f"SELECT id,name,gid,evidence,reference from disease where {' and '.join(qlist)} group by gid"
dcur.execute(sql)
dislist = dcur.fetchall()

# Extract disease names and gene list
gids = []
dids = {}
for dis in dislist:
  gids.append(f"'{dis[2].strip()}'")
  dids[dis[0]] = dis[1]

# Conversion to CE gene
gconv = {}
sql = f"SELECT gid1,gid2 from orthology where taxon1=6239 and gid2 in ({','.join(gids)}) UNION SELECT gid2,gid1 from orthology where taxon2=6239 and gid1 in({','.join(gids)})"
ocur.execute(sql)
orths = ocur.fetchall()
for ort in orths:
  if ort[1] in gconv:
    gconv[ort[1].strip()].append(ort[0].strip())
  else:
    gconv[ort[1].strip()] = [ort[0].strip()]

# Integrate
def integrator(list, elem, content):
  if content[0] in list:
    if elem in list[content[0]]:
      list[content[0]][elem].append(content[4])
    else:
      list[content[0]][elem] = [content[4]]
  else:
    list[content[0]] = {elem:[content[4]]}

disgene = {}
for dis in dislist:
  if re.match('^WBGene[0-9]+$', dis[2].strip()):
    integrator(disgene, dis[2].strip(), dis)
  else:
    if dis[2] in gconv:
      for gc in gconv[dis[2].strip()]:
        integrator(disgene, gc, dis)

# Output result
content = []
for did in disgene:
  for dg in disgene[did]:
    content.append(f'<tr><td style="border:1px solid black;">{dids[did]}</td><td style="border:1px solid black;">{dg}</td><td style="border:1px solid black;">{",".join(disgene[did][dg])}</td></tr>')
    #print(f"{dids[did]}\t{dg}\t{','.join(disgene[did][dg])}")
html = f'<table style="border-collapse: collapse;"><tr><th style="border:1px solid black;">Disease</th><th style="border:1px solid black;">CE Gene ID</th><th style="border:1px solid black;">Reference</th></tr>{"".join(content)}</table>'
display(HTML(html))

In [ ]:
#@title CE gene details
import sqlite3
geneid = '' #@param {type: 'string'}
sql = f"select * from gene where gid='{geneid}'"
cecur.execute(sql)
res = cecur.fetchall()
if len(res) == 0:
  print("Not found.")
else:
  print("Gene name:", res[0][1])
  print("Gene synonyms:", res[0][8])
  print("Gene description:", res[0][9])

# Reference
> The Alliance of Genome Resources: transforming comparative genomics. 2023. Bult CJ, Sternberg PW. Mamm Genome, 2023;34(4):531-44. doi: 10.1007/s00335-023-10015-2. PMID: 37666946.

> WormBase: a comprehensive resource for nematode research. Nucleic Acids Res. 38 (Database issue): D463–7. doi:10.1093/nar/gkp952. PMC 2808986. PMID 19910365.